dataset for box office regression

In [1]:
!pip list | grep "sesdiff\\|transformers"

sesdiff                   0.3.2
transformers              4.49.0


In [1]:
import pandas as pd
from pathlib import Path
from glob import glob
import os
import re
from sesdiff import shortest_edit_script, Mode

import os
import random
import functools
import csv
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import evaluate

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score

from scipy.stats import pearsonr
from datasets import Dataset, DatasetDict
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

from transformers import (
    #AutoModelForSequenceClassification,
    AutoTokenizer,
    #BitsAndBytesConfig,
    #TrainingArguments,
    #Trainer,
    #DataCollatorWithPadding
)
from transformers.models.llama import LlamaModel

In [2]:
folder = Path('storykg')
#file_ranking = '250331_최종데이터목록.xlsx'
file_ranking = 'ranking_list_20250403.xlsx'
df_ranking = pd.read_excel(folder/file_ranking, index_col=0)
df_ranking = df_ranking[['영화명','장르','전국관객수', '파일명']]

df_ranking = df_ranking.rename(
    columns = {
        '영화명': 'title',
        '장르': 'genre',
        '전국관객수': 'boxoffice',
        '파일명': 'filename',
    }
)

df_ranking['title'] = df_ranking['title'].apply(str)
df_ranking['filename'] = df_ranking['filename'].apply(str)

In [3]:
csv_list = sorted(glob(str(folder/'250331_디지털화시나리오_csv'/'*.csv')))
csv_list[0:5]

['storykg/250331_디지털화시나리오_csv/0001_명량.csv',
 'storykg/250331_디지털화시나리오_csv/0002_극한직업.csv',
 'storykg/250331_디지털화시나리오_csv/0003_신과함께_죄와벌.csv',
 'storykg/250331_디지털화시나리오_csv/0004_국제시장.csv',
 'storykg/250331_디지털화시나리오_csv/0004_파묘.csv']

In [4]:
replace_title = {
    '한산': '한산: 용의 출현',
    '노량': '노량: 죽음의 바다',
    '해무': '해무(海霧)',
    '탐정홍길동': '탐정 홍길동: 사라진 마을',
    '불한당': '불한당: 나쁜 놈들의 세상',
    '미확인동영상': '미확인 동영상 : 절대클릭금지',
    '만추': '만추 리마스터링',
    '협녀': '협녀, 칼의 기억',
    '미스터아이돌': 'Mr.아이돌',
    '심야카페': '심야카페: 미씽 허니',
}

In [5]:
def min_edit_distance(df, s):
    min_dist = 999999
    min_list = []
    for i, row in df.iterrows():
        t = row['title']
        t = re.sub('\ |\?|\.|\!|\/|\,|\:|\-|\_|\;', '', t)
        s = re.sub('\ |\?|\.|\!|\/|\,|\:|\-|\_|\;', '', s)
        result = shortest_edit_script(t, s)
        dist = result.distance()
        if dist < min_dist:
            min_dist = dist
            min_list = [i]
        elif dist == min_dist:
            min_list.append(i)
    return min_dist, min_list
    
for csv in csv_list[:]:
    base_name = os.path.basename(csv)
    title_csv = base_name[5:-4]
    if title_csv in replace_title:
        title_csv = replace_title[title_csv]
    #print(title_csv)
    dist, pos = min_edit_distance(df_ranking, title_csv)
    if dist > 0:
        print(dist, title_csv, base_name, 'not found')

    if dist == 0:
        df_ranking.loc[pos, 'filename'] = base_name
    '''
    ser = df_ranking['title'].str.contains(title_csv)
    found = len(df_ranking.loc[ser == True])
    if found == 0:
        print(title_csv)
    '''

2 스캔들 0103_스캔들.csv not found
4 째째한로맨스 0255_째째한로맨스.csv not found
2 아빠는여자를좋아해 1005_아빠는여자를좋아해.csv not found


In [13]:
df = df_ranking[df_ranking['filename'] != 'nan'].copy()
df['genre_ascat']=df['genre'].astype('category')
df['genre_category']=df['genre_ascat'].cat.codes

In [14]:
df

,title,genre,boxoffice,filename,genre_ascat,genre_category
순번,,,,,,
1,명량,사극,17613682,0001_명량.csv,사극,7
2,극한직업,코미디,16264944,0002_극한직업.csv,코미디,13
3,신과함께-죄와 벌,판타지,14410754,0003_신과함께_죄와벌.csv,판타지,14
4,국제시장,드라마,14245998,0004_국제시장.csv,드라마,3
5,베테랑,액션,13395400,0005_베테랑.csv,액션,10
...,...,...,...,...,...,...
1595,피터팬의 공식,드라마,2536,1716_피터팬의공식.csv,드라마,3
1604,가능한 변화들,드라마,2467,0209_가능한 변화들.csv,드라마,3
1644,가족시네마,드라마,2068,1329_가족시네마.csv,드라마,3


In [15]:
box_max = df['boxoffice'].max()
box_min = df['boxoffice'].min()
df['boxoffice_norm'] = (df['boxoffice']-box_min)/(box_max-box_min)
df

,title,genre,boxoffice,filename,genre_ascat,genre_category,boxoffice_norm
순번,,,,,,,
1,명량,사극,17613682,0001_명량.csv,사극,7,1.0
2,극한직업,코미디,16264944,0002_극한직업.csv,코미디,13,0.923426
3,신과함께-죄와 벌,판타지,14410754,0003_신과함께_죄와벌.csv,판타지,14,0.818155
4,국제시장,드라마,14245998,0004_국제시장.csv,드라마,3,0.808801
5,베테랑,액션,13395400,0005_베테랑.csv,액션,10,0.760509
...,...,...,...,...,...,...,...
1595,피터팬의 공식,드라마,2536,1716_피터팬의공식.csv,드라마,3,0.000136
1604,가능한 변화들,드라마,2467,0209_가능한 변화들.csv,드라마,3,0.000132
1644,가족시네마,드라마,2068,1329_가족시네마.csv,드라마,3,0.000109


In [26]:
csv_folder = folder/'250331_디지털화시나리오_csv'

In [16]:
import math
def extract_text(df):
    text = ''
    for i, row in df.iterrows():
        person = row['인물']
        dialog = row['지문/대사']
        if type(person) is str:
            text += person + ': '
        if type(dialog) is str:
            text += dialog + '\n'
    #print(text)
    return text

#s = extract_text(df_csv_scene)

In [17]:
model_name = "meta-llama/Meta-Llama-3-8B"

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

In [19]:
MAX_LEN = 2048

In [20]:
#inputs = tokenizer([s, s+'\nabc'], return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN)

In [21]:
#inputs

In [22]:
#len(inputs['input_ids'][0])

In [23]:
train_size = 0.9 # 80% of data
test_size = 0.1 # 20% of data
df_train, df_val = train_test_split(df, train_size=train_size, test_size=test_size, random_state=42)

In [27]:
def csv_to_texts(df):
    #n_scene = int(df['scene'].max())
    df['scene']=df['scene'].apply(str)
    df['scene_ascat']=df['scene'].astype('category')
    #df['scene_category'] = df['scene_ascat'].cat.codes
    #category_map = {code: category for code, category in enumerate(df['scene_ascat'].cat.categories)}
    category = [category for _, category in enumerate(df['scene_ascat'].cat.categories) ]
    category_sorted = []
    for i, row in df.iterrows():
        if row['scene_ascat'] not in category_sorted:
            category_sorted.append(row['scene_ascat'])
    #print(category)
    #print(category_sorted)
    df['scene_ascat'] = df['scene_ascat'].cat.reorder_categories(category_sorted)
    category_map = {code: category for code, category in enumerate(df['scene_ascat'].cat.categories)}
    category_code = [code for code, _ in enumerate(df['scene_ascat'].cat.categories) ]
    df['scene_category'] = df['scene_ascat'].cat.codes
    
    #print(category_code)
    #print(category_map)
    #print(n_scene)
    texts = []
    for scene in category_code:
        df_scene = df[df['scene_category'] == scene]
        #print('df_scene =', df_scene)
        text = extract_text(df_scene)
        #print(text)
        texts.append(text)
    return texts

def texts_to_ids(texts, max_len = MAX_LEN):
    n_scene = len(texts)
    ids = []
    for scene in range(n_scene):
        inputs = tokenizer(texts[scene], return_tensors="pt", padding=True, truncation=True, max_length=max_len)
        ids.append(inputs)
    return ids

def pack_ids(ids, max_len = MAX_LEN):
    n_ids = len(ids)
    packed_ids = []
    n_texts = []
    for i in range(n_ids):
        if i == 0:
            packed_ids.append(ids[i])
            n_texts.append(1)
        elif len(packed_ids[-1]['input_ids'][0]) + len(ids[i]['input_ids'][0]) <= max_len:
            packed_ids[-1]['input_ids'] = torch.unsqueeze(torch.cat( [packed_ids[-1]['input_ids'][0], ids[i]['input_ids'][0]]), dim=0)
            packed_ids[-1]['attention_mask'] = torch.unsqueeze(torch.cat( [packed_ids[-1]['attention_mask'][0], \
                                                                           ids[i]['attention_mask'][0]]), dim=0)
            n_texts[-1] += 1
        else:
            packed_ids.append(ids[i])
            n_texts.append(1)
    return packed_ids, n_texts

In [28]:
from IPython.display import display

def create_features(df):
    cols = list(df.reset_index().columns)
    #print(cols)

    df_new = pd.DataFrame(columns=cols)
    #display(df_new)
    all_packed_ids = []
    all_packed_texts = []
    for i, row in df.reset_index().iterrows():
        #display(row.to_frame().T)
        csv_file = row['filename']
        #if csv_file != '0188_브로커.csv':
        #if csv_file != '0864_파랑주의보.csv':
            #continue
        #print(csv_file)
        df_csv = pd.read_csv(csv_folder/csv_file)
        #display(df_csv)
        texts = csv_to_texts(df_csv)
        ids = texts_to_ids(texts)
        packed_ids, n_texts = pack_ids(ids)
        n_pack = len(packed_ids)
        print(n_pack, csv_file)

        all_packed_ids += packed_ids
        start_texts = 0
        for j in range(n_pack):
            df_new = pd.concat([df_new, row.to_frame().T], ignore_index=True)
            all_packed_texts.append(''.join(texts[start_texts:start_texts+n_texts[j]]))
            start_texts += n_texts[j]
    df_new['input'] = all_packed_texts
    return df_new, all_packed_ids
    
df_train_new, train_all_packed_ids = create_features(df_train)
df_train_new

16 0889_마요네즈.csv
20 0188_브로커.csv
27 0193_버닝.csv
28 0073_바람과 함께 사라지다.csv
15 0864_파랑주의보.csv
28 0181_내 사랑 내 곁에.csv
27 0187_헤어질 결심.csv
26 0161_블라인드.csv
22 0082_신라의달밤.csv
11 1366_어쩌면 우린 헤어졌는지 모른다.csv
16 1699_고양이장례식.csv
30 0331_임금님의사건수첩.csv
28 0128_프리즌.csv
10 0083_집으로.csv
26 0071_엽기적인그녀.csv
22 0532_허삼관.csv
29 0143_아수라.csv
26 0798_무뢰한.csv
22 0068_추격자.csv
22 1067_눈길.csv
23 0895_배심원들.csv
23 1244_후회하지않아.csv
25 0644_그날의분위기.csv
9 1716_피터팬의공식.csv
10 1643_마이제너레이션.csv
29 0879_신장개업.csv
20 1319_유포자들.csv
23 0015_부산행.csv
18 1541_아내의애인을만나다.csv
15 1039_웨딩드레스.csv
32 0019_기생충.csv
23 1534_돼지같은여자.csv
28 0859_허스토리.csv
24 0063_청년경찰.csv
41 0293_뺑반.csv
15 0460_돈의맛.csv
27 0858_맨발의꿈.csv
29 0096_역린.csv
43 0146_내 머리 속의 지우개.csv
13 1249_철없는아내와파란만장한남편그리고태권소녀.csv
25 0025_국가대표.csv
36 1162_돌려차기.csv
19 0869_좋지아니한가.csv
27 0131_극비수사.csv
24 0042_럭키.csv
26 0435_결혼전야.csv
28 1170_미스터아이돌.csv
29 0579_손님.csv
29 0288_코리아.csv
34 0279_오늘의연애.csv
16 1370_파란대문.csv
17 0841_싱글라이더.csv
29 0022_설국열차.csv
17 0849_산부인과.csv
29 0201_좋은놈나쁜놈이상한놈.csv
2

,순번,title,genre,boxoffice,filename,genre_ascat,genre_category,boxoffice_norm,input
0,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,"아정 아파트 앞\n마당을 쓸고 있는 경비.\n남편, 여행 가방을 끌고 해기 손을 잡..."
1,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,"아정 아파트 부엌 (저녁)\n아정, 엄마, 해기, 식사를 한다.\n엄마: (먹으며)..."
2,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,아정 아파트 내부 (오후)\n아정 아파트 외경 인서트\n빽빽이 널린 빨래.\n전화를...
3,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,"아정 아파트 내부 (오후)\n엄마, 곱게 차려입고 베란다 밖을 보고 있다.\n해기 ..."
4,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,"아정 아파트 내부 (낮)\n아파트 외경 인서트\n아정, 식탁에 앉아 일하고 있고 엄..."
...,...,...,...,...,...,...,...,...,...
10261,121,한반도,드라마,3880308,0115_한반도.csv,드라마,3,0.220294,상현의 집무실(밤)\n일기를 들고 있는 상현의 얼굴이 착잡하다. \n핸드폰을 꺼내 ...
10262,121,한반도,드라마,3880308,0115_한반도.csv,드라마,3,0.220294,대통령 집무실(밤)\n굳은 얼굴로 책상에 앉아 있는 총리. \n노크 소리와 동시에 ...
10263,121,한반도,드라마,3880308,0115_한반도.csv,드라마,3,0.220294,덕수궁 경내(새벽)\n유식과 소방대장에 붙잡혀 구덩이에서 끌려 나오는 민재.\n민재...
10264,121,한반도,드라마,3880308,0115_한반도.csv,드라마,3,0.220294,안가 거실(새벽)\n개조된 한옥 분위기의 실내. \n테이블 한가운데에는 발굴된 진짜...


In [29]:
train_all_packed_ids[0]

{'input_ids': tensor([[128000,  54059,  30381,  ..., 117264, 124230,    345]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}

In [30]:
df_val_new, val_all_packed_ids = create_features(df_val)
df_val_new

9 1700_요정.csv
24 0033_써니.csv
22 0122_방자전.csv
29 0070_가문의 영광.csv
21 0074_조선명탐정 각시투구꽃의 비밀.csv
26 0106_포화속으로.csv
27 0523_원더풀라디오.csv
23 0173_7광구.csv
24 0475_성난변호사.csv
24 0010_광해, 왕이 된 남자.csv
23 0865_사랑하기때문에.csv
31 0860_그해여름.csv
25 0545_넘버3.csv
11 1657_대관람차.csv
24 0078_도가니.csv
15 1406_무수단.csv
27 0030_히말라야.csv
26 0114_달마야놀자.csv
25 1289_수취인불명.csv
19 0168_달콤, 살벌한 연인.csv
24 0945_소울메이트.csv
20 0437_악녀.csv
24 0160_화이 괴물을 삼킨 아이.csv
24 0196_윤희에게.csv
29 0001_명량.csv
26 0910_가루지기.csv
30 0012_신과함께_인과연.csv
26 0088_7급 공무원.csv
14 1456_녹색의자.csv
27 0183_범죄의 재구성.csv
26 0482_베스트셀러.csv
25 0170_극락도 살인사건.csv
18 1327_거인.csv
26 0290_악마를보았다.csv
24 0055_공동경비구역 JSA.csv
25 0337_하울링.csv
33 0018_실미도.csv
16 1381_룸쉐어링.csv
19 0144_굿모닝 프레지던트.csv
28 0072_동갑내기 과외하기.csv
28 0039_터널.csv
17 1304_송어.csv
20 0163_맨발의 기봉이.csv
14 1711_리프레쉬.csv
27 0075_군도 민란의 시대.csv
26 1239_혈투.csv
8 1200_최면.csv
19 0085_용의자.csv


,순번,title,genre,boxoffice,filename,genre_ascat,genre_category,boxoffice_norm,input
0,1565,요정,드라마,2861,1700_요정.csv,드라마,3,0.000154,국밥집(낮)\n어느 허름한 식당 안. \n한산하다.\n추리한 차림의 한호철(남/40...
1,1565,요정,드라마,2861,1700_요정.csv,드라마,3,0.000154,짝태집 (밤)\n영란과 호철이 웃으며 대화 중이다. \n테이블에 쌓인 빈 막걸리 통...
2,1565,요정,드라마,2861,1700_요정.csv,드라마,3,0.000154,집 / 거실 (밤)\n석이 물끄러미 부부를 바라본다.\n맞은편의 부부는 죄인처럼 고...
3,1565,요정,드라마,2861,1700_요정.csv,드라마,3,0.000154,집 / 부엌 (밤)\n영란이 설거지를 하고 있다.\n반 쯤 누운 호철은 거실에서 과...
4,1565,요정,드라마,2861,1700_요정.csv,드라마,3,0.000154,좁은 골목 (낮)\n호철이 담배를 끄고 서둘러 카페로 향하는데 전화가 울린다.\n호...
...,...,...,...,...,...,...,...,...,...
1093,109,용의자,액션,4131248,0085_용의자.csv,액션,10,0.234541,"국정원 정팀장 사무실, 오후\n심과장의 안내로 정팀장의 사무실로 들어서는 경희.\n..."
1094,109,용의자,액션,4131248,0085_용의자.csv,액션,10,0.234541,"도로, 오후\n동철의 오토바이와 경찰차 사이의 추격전. \r\n회현고가에서부터 용산..."
1095,109,용의자,액션,4131248,0085_용의자.csv,액션,10,0.234541,"해주그룹 로비, 오후\r\n로비에서 층별 안내도를 보고 있는 동철. \r\n58층에..."
1096,109,용의자,액션,4131248,0085_용의자.csv,액션,10,0.234541,"비상계단-사장실, 오후\n동철 다시 계단을 올라가다가 김실장이 놓쳤던 권총을 집어든..."


In [32]:
df_train_new.to_excel('sequence_classification/'+f'boffice_train_new.xlsx', engine='xlsxwriter')
df_val_new.to_excel('sequence_classification/'+f'boffice_val_new.xlsx', engine='xlsxwriter')

In [25]:
#device_map="DDP" # for DDP and running with `accelerate launch test_sft.py`
device_map='auto' # for PP and running with `python test_sft.py`

if device_map == "DDP":
    device_string = PartialState().process_index
    device_map={'':device_string}

In [26]:
model = LlamaModel.from_pretrained(
    model_name,
    #quantization_config=quantization_config,
    num_labels=len(category_map),
    device_map=device_map,
)

model

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaModel(
  (embed_tokens): Embedding(128256, 4096)
  (layers): ModuleList(
    (0-31): 32 x LlamaDecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((4096,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding()
)

In [27]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False
model.config.pretraining_tp = 1

In [28]:
#outputs = model(**inputs)

In [29]:
#outputs['last_hidden_state'].shape

In [30]:
len(train_all_packed_ids)//4+1

2567

In [31]:
folder

PosixPath('storykg')

In [34]:
from tqdm.auto import tqdm, trange

def padding(batch_ids, max_length):
    max_len = 0
    for i, ids in enumerate(batch_ids):
        length = len(ids['input_ids'][0])
        if length > max_len:
            max_len = length
        
    for i, ids in enumerate(batch_ids):
        length = len(ids['input_ids'][0])
        new_ids = torch.nn.functional.pad(ids['input_ids'][0], (0,max_len-length), mode='constant', value=tokenizer.pad_token_id)
        new_attention = torch.nn.functional.pad(ids['attention_mask'][0], (0,max_len-length), mode='constant', value=0)
        #print(new_ids.shape)
        #print(new_ids)
        if i == 0:
            padded_ids = torch.unsqueeze(new_ids, dim=0)
            padded_attention = torch.unsqueeze(new_attention, dim=0)
        else:
            padded_ids = torch.cat([padded_ids, torch.unsqueeze(new_ids, dim=0)])
            padded_attention = torch.cat([padded_attention, torch.unsqueeze(new_attention, dim=0)])
    return {'input_ids': padded_ids, 'attention_mask': padded_attention }
        

def make_predictions(model, df, packed_ids, prefix=''):

  # Convert summaries to a list
  #sentences = df.input.tolist()

  # Define the batch size
  batch_size = 4  # You can adjust this based on your system's memory capacity

  # Initialize an empty list to store the model outputs
  all_outputs = []

  #print('batch_size =', batch_size)
  # Process the sentences in batches
  for i in trange(0, len(packed_ids), batch_size):
      # Get the batch of sentences
      batch_ids = packed_ids[i:i+batch_size]

      # Tokenize the batch
      #inputs = tokenizer(batch_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)
      inputs = padding(batch_ids, max_length=MAX_LEN)

      # Move tensors to the device where the model is (e.g., GPU or CPU)
      inputs = {k: v.to('cuda' if torch.cuda.is_available() else 'cpu') for k, v in inputs.items()}
      #print(inputs)

      # Perform inference and store the logits
      with torch.no_grad():
          outputs = model(**inputs)
          #print(outputs.keys())
          last_hidden_state = outputs['last_hidden_state'].cpu()
          non_pad_mask = (inputs['input_ids'] != model.config.pad_token_id)
          sequence_len = torch.sum(non_pad_mask, dim=1).cpu().numpy()
          #print('non_pad_mask =', non_pad_mask)
          #print('sequence_len =', sequence_len)
          #print(last_hidden_state.shape)
          vectors = [ last_hidden_state[i][:n].clone().detach() for i, n in enumerate(sequence_len)] 
          #print(vectors)
          #print(last_hidden_state)
          #all_outputs += vectors
          #for k in range(batch_size):
              #df.loc[i+k, 'embed'] = last_hidden_state
          #all_outputs.append(outputs['logits'])
          for j, vector in enumerate(vectors):
              torch.save(vector, Path('sequence_classification')/prefix/f'{i+j:05d}.pt')
    
  return all_outputs

  #final_outputs = torch.cat(all_outputs, dim=0)
  #df['predictions']=final_outputs.argmax(axis=1).cpu().numpy()
  #df['predictions']=df['predictions'].apply(lambda l:category_map[l])

#outputs = make_predictions(model, df_train, train_all_packed_ids, prefix='train')


In [33]:
outputs = make_predictions(model, df_train, train_all_packed_ids, prefix='train')

  0%|          | 0/2567 [00:00<?, ?it/s]

NameError: name 'all_outputs' is not defined

In [35]:
outputs = make_predictions(model, df_val, val_all_packed_ids, prefix='val')

  0%|          | 0/275 [00:00<?, ?it/s]

In [42]:
#!pip install xlsxwriter
%pip list | grep xlsxwriter

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [46]:
def save_df_ids(df, packed_ids, prefix=''):
    df.to_excel('sequence_classification/'+f'genre_{prefix}.xlsx')
    torch.save(packed_ids, 'sequence_classification/'+f'genre_{prefix}.pt')
save_df_ids(df_train, train_all_packed_ids, prefix='train')
df_train_new.to_excel('sequence_classification/'+f'genre_train_new.xlsx', engine='xlsxwriter')
save_df_ids(df_val, val_all_packed_ids, prefix='val')
df_val_new.to_excel('sequence_classification/'+f'genre_val_new.xlsx', engine='xlsxwriter')

In [48]:
vector = torch.load('sequence_classification/train/00000.pt')

In [53]:
vector[0].type()

'torch.FloatTensor'

,순번,title,genre,boxoffice,filename,genre_ascat,genre_category,boxoffice_norm,input
0,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,"아정 아파트 앞\n마당을 쓸고 있는 경비.\n남편, 여행 가방을 끌고 해기 손을 잡..."
1,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,"아정 아파트 부엌 (저녁)\n아정, 엄마, 해기, 식사를 한다.\n엄마: (먹으며)..."
2,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,아정 아파트 내부 (오후)\n아정 아파트 외경 인서트\n빽빽이 널린 빨래.\n전화를...
3,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,"아정 아파트 내부 (오후)\n엄마, 곱게 차려입고 베란다 밖을 보고 있다.\n해기 ..."
4,820,마요네즈,가족,297213,0889_마요네즈.csv,가족,1,0.016866,"아정 아파트 내부 (낮)\n아파트 외경 인서트\n아정, 식탁에 앉아 일하고 있고 엄..."
...,...,...,...,...,...,...,...,...,...
10261,121,한반도,드라마,3880308,0115_한반도.csv,드라마,3,0.220294,상현의 집무실(밤)\n일기를 들고 있는 상현의 얼굴이 착잡하다. \n핸드폰을 꺼내 ...
10262,121,한반도,드라마,3880308,0115_한반도.csv,드라마,3,0.220294,대통령 집무실(밤)\n굳은 얼굴로 책상에 앉아 있는 총리. \n노크 소리와 동시에 ...
10263,121,한반도,드라마,3880308,0115_한반도.csv,드라마,3,0.220294,덕수궁 경내(새벽)\n유식과 소방대장에 붙잡혀 구덩이에서 끌려 나오는 민재.\n민재...
10264,121,한반도,드라마,3880308,0115_한반도.csv,드라마,3,0.220294,안가 거실(새벽)\n개조된 한옥 분위기의 실내. \n테이블 한가운데에는 발굴된 진짜...
